In [ ]:
import numpy as np
import codecs

In [ ]:
file_path = './input.txt'
data = codecs.open(file_path, 'r', 'utf-8' , errors='ignore').read()
print data[:20]
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print 'text has %d characters, %d unique'%(data_size, vocab_size)
char_to_ix = {ch:i for i,ch in enumerate(chars)}
ix_to_char = {i:ch for i,ch in enumerate(chars)}

In [78]:
# hyperparameters
hidden_size = 100
seq_length = 10
learning_rate = 1e-1

# model parameters

Wxh = np.random.randn(hidden_size, vocab_size)*0.01
Whh = np.random.randn(hidden_size, hidden_size)*0.01
Why = np.random.randn(vocab_size, hidden_size)*0.01
bh = np.zeros((hidden_size, 1))
by = np.zeros((vocab_size, 1))

In [90]:
def lossFun(inputs, targets, hprev):

    xs, hs, ys, ps = {}, {}, {}, {}
    hs[-1] = np.copy(hprev)
    loss = 0

    for t in xrange(len(inputs)):
        xs[t] = np.zeros((vocab_size, 1))
        xs[t][inputs[t]] = 1
        hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh)
        ys[t] = np.dot(Why, hs[t]) + by
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t]))
        loss += -np.log(ps[t][targets[t], 0])
    # backward pass: computer gradients 
    dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
    dbh, dby = np.zeros_like(bh), np.zeros_like(by)
    dhnext = np.zeros_like(hs[0])
    for t in reversed(xrange(len(inputs))):
        dy = np.copy(ps[t])
        dy[targets[t]] -= 1
        dWhy += np.dot(dy, hs[t].T)
        dby += dy
        dh = np.dot(Why.T, dy) + dhnext
        dhraw = (1 - hs[t] * hs[t]) * dh
        dbh += dhraw
        dWxh += np.dot(dhraw, xs[t].T)
        dWhh += np.dot(dhraw, hs[t-1].T)
        dhnext = np.dot(Whh.T, dhraw)
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        np.clip(dparam, -5, 5, out=dparam)
    return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

In [88]:
def sample(h, seed_ix, n):
    x = np.zeros((vocab_size, 1))
    x[seed_ix] = 1
    ixes = []
    for t in xrange(n):
        h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
        y = np.dot(Why, h) + by
        p = np.exp(y)/np.sum(np.exp(y))
        ix = np.random.choice(range(vocab_size), p=p.ravel())
        x = np.zeros((vocab_size, 1))
        x[ix] = 1
        ixes.append(ix)
    return ixes

In [80]:
n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh),np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by)
smooth_loss = -np.log(1.0/vocab_size)*seq_length

In [92]:
while True:
    if p+seq_length+1 >= len(data) or n == 0:
        hprev = np.zeros((hidden_size, 1))
        p = 0
    inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
    targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

    # sample from the model now and then
    if n % 100 == 0:
        sample_ix = sample(hprev, inputs[0], 200)
        txt = ''.join(ix_to_char[ix] for ix in sample_ix)
        print '------\n %s \n-------'%(txt, )
    
    # forward seq_length characters through the net and fetch gradient
    loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001
    if n % 100 == 0: print 'iter: %d, loss: %f'%(n, smooth_loss)
    
    # perform paramter update with Adagrad
    for param, dparam, mem in zip([Wxh, Whh, Why, bh, by],
                                  [dWxh, dWhh, dWhy, dbh, dby],
                                  [mWxh, mWhh, mWhy, mbh, mby]):
        mem += dparam * dparam
        param += -learning_rate * dparam / np.sqrt(mem + 1e-8)
    
    p += seq_length
    n += 1

------
 果果果果果果
果君果怕给国怕果果果果果定果很果果果果果现果果果果果果果果果果果果每果果果果果很果果果果很很果果果果果果果果果果果果果果果果你果果果怕果很果果果果果果果果果果果很果果果果果果果果却果果果果果
果果果小结果果果果果果果
果果果果果果果果果果果果果果果果果果果见果果果果满果果果果现果果果果果果果果果很果果果结果果果果很果果却果果果果怕由果果果果果果果流果果那果果果有很果果果果果果果怕果果 
-------
iter: 100, loss: 75.041156
------
 必须坚须
须必须坚强必须坚强坚掉必强必强坚强必须坚须坚伤必强坚须坚须必须坚须坚强坚强坚到我须坚须
强必强必须坚须坚强坚须坚强必须坚强必强必须坚强像须坚强坚须坚强坚强坚强坚须坚须坚我坚须必无必须坚强坚须必须坚强坚须必须坚过必须必须必须坚强坚须必须坚强必须必须们望坚须坚我坚强坚须必须坚须必强必须必须必须坚强必须坚须坚须必强坚须必强坚强必强坚须坚须必须坚须坚沽坚强
须坚须坚强在须坚强坚强坚须必强坚须坚须 
-------
iter: 200, loss: 73.728315
------
 今睡中时安的口的有雨今睡今雨它睡今睡拔的善长巨没希雨今睡响
地穿今里怕着-里今阳里新今

们机雨得起实雨什希界雨一雨宜真去国去睡我路里无今我那睡滚
中晚-的中睡堪雨今睡为福明望今睡的的的破玩地-雨今睡 歌游滚今中望些的 今的生睡口过缈谈今紧后的我睡今去希雨今 
样亮后今重和的中睡今雨中睡爱勤有雨生雨今茫睁音车睡今来希的今睡叫雨良那今争希口今善仿歌子雨国多
且 雨年的去睡巨-国瞬彩彩今看光睡今去冷追 
-------
iter: 300, loss: 72.241414
------
 发所
次发批断着吧血产在望了一唐
情件首
这我旁
到-生星
个这在有产批过生产的像火
霓个生游这个生明批人么

发了
这发握把批丝样 的把希
望像美收渺
郁 的个驱
别人为着到
好人怎 
向姐个
后自产-什全你的撒佛
怎埃寂里这产它石上口到鸟上个晚产-误游一了
失个岗
强你可个想点-什追个生
生冷紧发生放有个如爷的歌批今生个飘什道
生那是发批把没道经难这那生个生着批个永看里希的亮讲
是天了过么去批 
-------
iter: 400, loss: 71.493258
------
 不烂飞眼


有感拂我得

KeyboardInterrupt: 